In [ ]:
pip install pandas

In [ ]:
pip install bs4

In [91]:
pip install dateparser

     |████████████████████████████████| 294 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 777 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 1.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [42]:
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re
import json
import requests

In [43]:
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Instantiate options
options = Options()

# Add extra options
options.add_argument("--window-size=1920,1080")  # Set the window size
options.add_argument("--disable-infobars")  # Disable the infobars
options.add_argument("--disable-popup-blocking")  # Disable pop-ups
options.add_argument("--ignore-certificate-errors")  # Ignore certificate errors
options.add_argument('--headless') # ensure GUI is off
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Set path to chromedriver as per your configuration
homedir = "fabiodc"
options.binary_location = f"/home/{homedir}/chrome-linux64/chrome"
webdriver_service = Service(f"/home/{homedir}/chromedriver-linux64/chromedriver")

# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=options)

In [3]:

db_hash = {}

In [45]:
import requests

try:
    public_ip = requests.get('https://api.ipify.org').text
    print(f"Public IP Address: {public_ip}")
except requests.RequestException as e:
    print(f"Error retrieving public IP address: {e}")

Public IP Address: 204.216.221.185


In [46]:
def get_page_source(url):
  driver.get(url)
  return driver.page_source

In [47]:
def get_page_source_with_retries(url, max_retries=3, delay=3):
    retries = 0
    html = None
    while retries < max_retries:
        try:
            html = get_page_source(url)
            break  # If successful, exit the loop
        except Exception as e:
            print(f"Attempt {retries+1}: An exception occurred: {e}")
            time.sleep(delay)  # Wait for the specified delay
            retries += 1
    return html

In [11]:
url_first = 'https://www.amazon.it/deals?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
#page is a mutiple of 6 starting from 0, so page 1 is '' page 2 is '6' page 3 is '12' and so on
page = '6' #page1
url_last = '0%252C%2522presetId%2522%253A%2522deals-collection-lightning-deals%2522%252C%2522dealState%2522%253A%2522UPCOMING%2522%252C%2522dealType%2522%253A%2522LIGHTNING_DEAL%2522%252C%2522sorting%2522%253A%2522FEATURED%2522%257D'
url = url_first + page + url_last
#driver.get(url)
#html = driver.page_source
html = get_page_source(url)

In [12]:
pattern = r'\{"widgetId".*\}\)\n '
match = re.search(pattern, html)

In [13]:
deals_data = match.group().rstrip(")\n ")

In [14]:
# i'm getting less result for some reason both using selenium or requests, maybe is based on ip?
# it's confirmed that i get different offers based on ip, i need an italian ip
json.loads(deals_data)['prefetchedData']['aapiSearchDealsTotalCount']

234

In [71]:
db_hash

{'B07G88LD1V': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'A2PR9V5I6YJF94', 'A35NQTY9UZD19J'}},
 'B07NRR44DZ': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'ADUBRBI0ZEPGX'}},
 'B0C5J94DNC': {'start_time': '2024-04-29T05:30:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'AMJ3X9H5D00X1'}},
 'B082VV6FMJ': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': set()},
 'B0BN7X7Q8H': {'start_time': '2024-04-29T06:05:00.000Z',
  'actual_start_time': '2024-04-29T05:35:00.000Z',
  'promo_codes': set()},
 'B09SCQY1CB': {'start_time': '2024-04-29T05:40:00.000Z',
  'actual_start_time': '2024-04-29T05:40:00.000Z',
  'promo_codes': set()},
 'B0BTXTP442': {'start_time': '2024-04-29T05:40:00.000Z',
  'actual_start_time': '2024-04-29T05:40:00.000Z',
  'promo_codes': set()},
 'B0BLH

In [19]:
def parse_html_deals_page(html, db_hash):
  # Search for the pattern in the script content
  pattern = r'\{"widgetId".*\}\)\n '
  match = re.search(pattern, html)

  try:
    deals_data = match.group().rstrip(")\n ")
    deals = json.loads(deals_data)['prefetchedData']['aapiGetDealsList'][0]['entities']
  except Exception as e:
    # if we get to this point it should mean that we have scraped all the deals or that we have been blocked
    return

  scraped_deals = {}
  for deal in deals:
    product_id=deal['entity']['details']['entity']['landingPage']['url'].split("/")[-1]
    # this is stupid, i need to improve this logic
    if product_id in db_hash:
      #print("already_present")
      scraped_deals[product_id] = db_hash[product_id]
      continue
    start_time= deal['entity']['details']['entity']['startTime']['value']
    #start_time dosen't count early offers for prime members
    time_dic = deal['entity']['badge']['entity']['messaging']['content']['fragments'][1]
    promo_codes = get_promo_ids("https://www.amazon.it/dp/"+product_id)
    actual_start_time= time_dic['dateTime'] if 'dateTime' in time_dic else time_dic['countdownTimer']['targetTime']
    print(product_id + " : " + start_time + " : " + actual_start_time + " " + " ".join(promo_codes))
    scraped_deals[product_id] = {"start_time": start_time, "actual_start_time":actual_start_time, "promo_codes": promo_codes}

  return scraped_deals

In [121]:
def get_promo_ids(url):
  session = requests.Session()
  session.mount("https://", requests.adapters.HTTPAdapter(max_retries=100))
  session.mount("http://", requests.adapters.HTTPAdapter(max_retries=100))
  res = session.get(url)
  #to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
  html = res.content.decode(res.encoding)
  blocked_pattern = "api-services-support@amazon.com"
  if blocked_pattern in html:
    #blocked by amazon
    return get_promo_ids(url)
  promo_pattern = r'/promo/[^"]*'
  promos = re.findall(promo_pattern, html)
  return set([promo.split("/")[-1] for promo in promos])

In [65]:
def get_promo_data(url):
  res = requests.get(url)
  #to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
  html = res.content.decode(res.encoding)
  blocked_pattern = "api-services-support@amazon.com"
  if blocked_pattern in html:
    #blocked by amazon
    return get_promo_ids(url)
  promo_pattern = r'/promo/[^"]*'
  promos = re.findall(promo_pattern, html)
  promo_codes = set([promo.split("/")[-1] for promo in promos])
  price_pattern = r'\"price\"\:\"[^"]*\"'
  price = re.search(price_pattern, html).group().split(":")[-1].replace("\"", "")
  return {"price": float(price), "promo_codes": promo_codes}

In [69]:
get_promo_data("https://www.amazon.it/dp/B0CKXMKRCS?th=1")

{'price': 49.99, 'promo_codes': {'A2396UT407D7DX', 'A34YL5LFNE1RLD'}}

In [ ]:
html = driver.page_source
parse_html_deals_page(html,db_hash)

In [122]:
url_first = 'https://www.amazon.it/deals?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
#url_first = 'https://www.amazon.it/events/springdealdays?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
page = '' # first page
url_last = '0%252C%2522presetId%2522%253A%2522deals-collection-lightning-deals%2522%252C%2522dealState%2522%253A%2522UPCOMING%2522%252C%2522dealType%2522%253A%2522LIGHTNING_DEAL%2522%252C%2522sorting%2522%253A%2522FEATURED%2522%257D'
retry = 0
while True:
  url = url_first + page + url_last
  html = get_page_source_with_retries(url)
  # deals it's an hash where each key is a product_id
  deals = parse_html_deals_page(html,db_hash)
  if not deals:
    if (retry>2):
      break
    else:
      retry+=1
      # go on without incrementing page
      continue
  if deals:
    retry = 0
    for deal in deals:
      if deal not in db_hash:
        db_hash[deal] = deals[deal]
    page = str(int(page) + 1) if page.isdigit() else '1'
    print("page " + page)
  else:
    break

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
B0CPBHW9TN : 2024-04-30T08:05:00.000Z : 2024-04-30T07:35:00.000Z 
page 12
B06WD5H59M : 2024-04-30T07:35:00.000Z : 2024-04-30T07:35:00.000Z A36ZVRQO3N3ABW
B07VDKPDSC : 2024-04-30T07:40:00.000Z : 2024-04-30T07:40:00.000Z 
page 13
B0BLYNWS9R : 2024-04-30T08:10:00.000Z : 2024-04-30T07:40:00.000Z 
B0CC8Z2PZQ : 2024-04-30T07:40:00.000Z : 2024-04-30T07:40:00.000Z 
B0C66D2QSK : 2024-04-30T08:15:00.000Z : 2024-04-30T07:45:00.000Z 
page 14
B085STS7RG : 2024-04-30T08:20:00.000Z : 2024-04-30T07:50:00.000Z 
page 15
B08SJJFHDT : 2024-04-30T08:30:00.000Z : 2024-04-30T08:00:00.000Z 
page 16
page 17
B0CN933LVK : 2024-04-30T08:45:00.000Z : 2024-04-30T08:15:00.000Z 
page 18
B0CQ4L5QTL : 2024-04-30T08:45:00.000Z : 2024-04-30T08:15:00.000Z A168J7OAP2B8FE
B0BR7R5Q9M : 2024-04-30T08:45:00.000Z : 2024-04-30T08:15:00.000Z A26JJGEPJPVVJM
page 19
B0C4B11JKX : 2024-04-30T08:25:00.000Z : 2024-04-30T08:25:00.000Z 
B0CMBKHV6W : 2024-04-30

In [21]:
len(db_hash)

234

In [123]:
import pickle

# Store data (serialize)
with open('db_hash.pickle', 'wb') as handle:
    pickle.dump(db_hash, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [124]:
import pickle

with open('db_hash.pickle', 'rb') as handle:
    db_hash = pickle.load(handle)

len(db_hash)

610

AttributeError: 'dict' object has no attribute 'first'

In [24]:
db_hash

{'B07G88LD1V': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'A2PR9V5I6YJF94', 'A35NQTY9UZD19J'}},
 'B07NRR44DZ': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'ADUBRBI0ZEPGX'}},
 'B0C5J94DNC': {'start_time': '2024-04-29T05:30:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': {'AMJ3X9H5D00X1'}},
 'B082VV6FMJ': {'start_time': '2024-04-29T06:00:00.000Z',
  'actual_start_time': '2024-04-29T05:30:00.000Z',
  'promo_codes': set()},
 'B0BN7X7Q8H': {'start_time': '2024-04-29T06:05:00.000Z',
  'actual_start_time': '2024-04-29T05:35:00.000Z',
  'promo_codes': set()},
 'B09SCQY1CB': {'start_time': '2024-04-29T05:40:00.000Z',
  'actual_start_time': '2024-04-29T05:40:00.000Z',
  'promo_codes': set()},
 'B0BTXTP442': {'start_time': '2024-04-29T05:40:00.000Z',
  'actual_start_time': '2024-04-29T05:40:00.000Z',
  'promo_codes': set()},
 'B0BLH

In [113]:
import dateparser
def get_promo_details(promo_id):
  base_promo_url = 'https://www.amazon.it/promotion/psp/'

  promo_url = base_promo_url + promo_id
  html = requests.get(promo_url).text
  pattern = r'let getCartItemsRequest = ({.*?});'
  search = re.search(pattern, html, re.DOTALL)
  if not search:
    # if html contains 'La promozione è terminata' return kind amount and value 0, and end date 01/01/1970
    if 'La promozione è terminata' in html:
      return {"kind": "amount", "value": 0, "expired": True}
    else:
      return get_promo_details(promo_id)
  promo_data = search.group(1)
    # Try to convert the JSON string to a dictionary.
  promo_data = re.sub(r':\s*JSON\.stringify\(\w+\)', ': ""', promo_data)
  promo_dict = json.loads(promo_data)
  percent = promo_dict['benefitPercent'].lower()
  amount = promo_dict['benefitAmount']
 # if amount is valorized return {kind: "amount", value: amount} else return {kind: "percent", value: percent}
  return {"kind": "amount", "value": float(amount), "expired": False} if amount else {"kind": "percent", "value": float(percent), "expired": False}


In [110]:
get_promo_details("A2396UT407D7DX")

A2396UT407D7DX


{'kind': 'percent', 'value': 10.0, 'expired': False}

In [111]:
promo_db = {}

In [41]:
# pickle promo_db
with open('promo_db.pickle', 'wb') as handle:
    pickle.dump(promo_db, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
with open('promo_db.pickle', 'rb') as handle:
    promo_db = pickle.load(handle)

In [59]:
# search from db_hash wich key has 'A24P2TS1SMLXH6' between it's promo_codes
for key in db_hash:
  if 'A3EHCKLPXQM553' in db_hash[key]['promo_codes']:
    print(db_hash[key])
    print(key)

{'start_time': '2024-04-29T06:55:00.000Z', 'actual_start_time': '2024-04-29T06:25:00.000Z', 'promo_codes': {'A3EHCKLPXQM553'}}
B0CKLKCYJ8


In [125]:
for deal in db_hash:
  for promo in db_hash[deal]['promo_codes']:
    if promo not in promo_db:
      promo_db[promo] = get_promo_details(promo)
      print(promo + " : " + str(promo_db[promo]))

A2W229TM7FBV4O : {'kind': 'percent', 'value': 10.0, 'expired': False}
A1CMF77N4YAPLQ : {'kind': 'percent', 'value': 20.0, 'expired': False}
A168J7OAP2B8FE : {'kind': 'amount', 'value': 10.0, 'expired': False}
A26JJGEPJPVVJM : {'kind': 'amount', 'value': 15.0, 'expired': False}
AOY6QGLKV9NGJ : {'kind': 'percent', 'value': 5.0, 'expired': False}
A2SGB19PH7GUHK : {'kind': 'percent', 'value': 15.0, 'expired': False}
A25KQX7PIC1ULY : {'kind': 'percent', 'value': 15.0, 'expired': False}
AE0BPE9M94Q4A : {'kind': 'percent', 'value': 20.0, 'expired': False}
A3OQVZWXU783X8 : {'kind': 'percent', 'value': 10.0, 'expired': False}
A1UG9HU0UIG494 : {'kind': 'amount', 'value': 2.0, 'expired': False}
A1H4ZXYY5TJFZJ : {'kind': 'percent', 'value': 5.0, 'expired': False}
AQB9LWEDQ1X59 : {'kind': 'percent', 'value': 10.0, 'expired': False}
A28754U6INIW7H : {'kind': 'percent', 'value': 10.0, 'expired': False}
A26PP2FA65XWPX : {'kind': 'percent', 'value': 5.0, 'expired': False}
AY566J8LSWK1G : {'kind': 'perc

In [37]:
# convert all the values of the promo_db to decimal
for promo in promo_db:
    promo_db[promo]['value'] = float(promo_db[promo]['value'])

In [82]:
promo_db

{'A35NQTY9UZD19J': {'kind': 'percent', 'value': 8.0},
 'A2L0ZGX7K646U8': {'kind': 'percent', 'value': 8.0},
 'A15QMLOZ4X3ENV': {'kind': 'percent', 'value': 8.0},
 'A1RAK9BIFFWGR7': {'kind': 'percent', 'value': 8.0},
 'A3U5XZ8F7XORA5': {'kind': 'percent', 'value': 8.0},
 'A3G97Z1PAZ3HNK': {'kind': 'percent', 'value': 8.0},
 'ARQ2AC0SEEDQJ': {'kind': 'percent', 'value': 8.0},
 'A35QYZR7YQ9QIH': {'kind': 'percent', 'value': 7.0},
 'A3PES2DIT7FKRM': {'kind': 'percent', 'value': 6.0},
 'AR60RRPSCR0OQ': {'kind': 'percent', 'value': 50.0},
 'A3U4CD72MSRPZZ': {'kind': 'percent', 'value': 50.0},
 'A2AN4HH08QS8RW': {'kind': 'percent', 'value': 50.0},
 'A36ZVRQO3N3ABW': {'kind': 'percent', 'value': 5.0},
 'A3A9M0UBONHAVI': {'kind': 'percent', 'value': 5.0},
 'A1H2EAMAR4MQBK': {'kind': 'percent', 'value': 5.0},
 'A1Y4E25OWQIHZA': {'kind': 'percent', 'value': 5.0},
 'A2RNIAYCY3UHFH': {'kind': 'percent', 'value': 5.0},
 'A11NIR8WAT9X5': {'kind': 'percent', 'value': 5.0},
 'A35X6IWOHALUJ2': {'kind': 

In [115]:
# sort promo_db by value descending, returning before all the promos with kind percent and then the ones with kind amount, values should be considered as float, ignore expired promo
sorted_promo_db = {k: v for k, v in sorted(promo_db.items(), key=lambda item: (item[1]['kind'], item[1]['value']), reverse=True) if not v['expired']}
#print sorted promod db line by line
for promo in sorted_promo_db:
  print(promo + " : " + str(sorted_promo_db[promo]))


A2WSHVG0W5T8JN : {'kind': 'percent', 'value': 100.0, 'expired': False}
A25BT8U2WBHM7T : {'kind': 'percent', 'value': 100.0, 'expired': False}
A2BVKSI8NCXDBS : {'kind': 'percent', 'value': 100.0, 'expired': False}
A2LA4Y1ZAE7GU0 : {'kind': 'percent', 'value': 100.0, 'expired': False}
A2AN4HH08QS8RW : {'kind': 'percent', 'value': 50.0, 'expired': False}
A38APJBKCZ6TM4 : {'kind': 'percent', 'value': 50.0, 'expired': False}
A2AOCZJDIKK33Y : {'kind': 'percent', 'value': 45.0, 'expired': False}
A3EHCKLPXQM553 : {'kind': 'percent', 'value': 40.0, 'expired': False}
A1XBY64L2BTZ3X : {'kind': 'percent', 'value': 40.0, 'expired': False}
A1NKCP2KIGZR3R : {'kind': 'percent', 'value': 38.0, 'expired': False}
A3LW3M9ANVA9BM : {'kind': 'percent', 'value': 34.0, 'expired': False}
A14ITI4O371JL : {'kind': 'percent', 'value': 30.0, 'expired': False}
A2Q9RCCTP56W02 : {'kind': 'percent', 'value': 30.0, 'expired': False}
A3N3TH31HHCEUH : {'kind': 'percent', 'value': 30.0, 'expired': False}
AX2L5DJB3XBJ6 : {

In [116]:
# from db hash get all the values that have more than one promo code of kind percent, with percent > 5, print the deal id, the deal data and the promo codes percents
for deal in db_hash:
  percent_promos = [promo for promo in db_hash[deal]['promo_codes'] if promo_db[promo]['kind'] == 'percent' and promo_db[promo]['value'] > 5 and not promo_db[promo]['expired']]
  if len(percent_promos) > 1:
    print(deal + " : " + str(db_hash[deal]) + " : " + str([promo_db[promo]['value'] for promo in percent_promos]))

B0B5LDJ5JB : {'start_time': '2024-04-29T06:50:00.000Z', 'actual_start_time': '2024-04-29T06:20:00.000Z', 'promo_codes': {'A19K8KHEJAYYE3', 'A2WSHVG0W5T8JN', 'A35QYZR7YQ9QIH'}} : [100.0, 7.0]
B0CKXMKRCS : {'start_time': '2024-04-29T06:25:00.000Z', 'actual_start_time': '2024-04-29T06:25:00.000Z', 'promo_codes': {'A2396UT407D7DX', 'A34YL5LFNE1RLD'}} : [10.0, 15.0]
B0BXDMZY7Q : {'start_time': '2024-04-29T07:00:00.000Z', 'actual_start_time': '2024-04-29T06:30:00.000Z', 'promo_codes': {'A1CUERVNCL49CK', 'A2L0ZGX7K646U8'}} : [10.0, 8.0]
B09MJ2KSS1 : {'start_time': '2024-04-29T08:50:00.000Z', 'actual_start_time': '2024-04-29T08:20:00.000Z', 'promo_codes': {'A2WSHVG0W5T8JN', 'A35QYZR7YQ9QIH'}} : [100.0, 7.0]
B0B5HGZ7X5 : {'start_time': '2024-04-29T09:20:00.000Z', 'actual_start_time': '2024-04-29T08:50:00.000Z', 'promo_codes': {'A2WSHVG0W5T8JN', 'A35QYZR7YQ9QIH'}} : [100.0, 7.0]
B0B4WLJVWH : {'start_time': '2024-04-29T09:30:00.000Z', 'actual_start_time': '2024-04-29T09:00:00.000Z', 'promo_codes'

In [129]:
import pytz
# from db hash get all the values that have at least one promo code of kind percent, with percent >= 30 and, print the deal id, the deal data and the promo codes percents and the amazon link to product "amazon.it/dp/product_id" if the deal acutal_start_time it's not in the past
for deal in db_hash:
    percent_promos = [promo for promo in db_hash[deal]['promo_codes'] if promo_db[promo]['kind'] == 'percent' and promo_db[promo]['value'] >= 30 and not promo_db[promo]['expired']]
    if percent_promos and dateparser.parse(db_hash[deal]['actual_start_time']) > datetime.now(pytz.utc):
        print(deal + " : " + str(db_hash[deal]) + " : " + str([promo_db[promo]['value'] for promo in percent_promos]) + " : " + "https://www.amazon.it/dp/" + deal)

B0BWXD3N1N : {'start_time': '2024-04-30T06:05:00.000Z', 'actual_start_time': '2024-04-30T05:35:00.000Z', 'promo_codes': {'A1XBY64L2BTZ3X'}} : [40.0] : https://www.amazon.it/dp/B0BWXD3N1N
B083TXPZ6H : {'start_time': '2024-04-30T06:20:00.000Z', 'actual_start_time': '2024-04-30T05:50:00.000Z', 'promo_codes': {'A14ITI4O371JL'}} : [30.0] : https://www.amazon.it/dp/B083TXPZ6H
B08TRCXB2J : {'start_time': '2024-04-30T06:20:00.000Z', 'actual_start_time': '2024-04-30T05:50:00.000Z', 'promo_codes': {'A1NKCP2KIGZR3R'}} : [38.0] : https://www.amazon.it/dp/B08TRCXB2J
B0BNVWXLDR : {'start_time': '2024-04-30T06:25:00.000Z', 'actual_start_time': '2024-04-30T05:55:00.000Z', 'promo_codes': {'A2Q9RCCTP56W02'}} : [30.0] : https://www.amazon.it/dp/B0BNVWXLDR
B0CB5J5GCV : {'start_time': '2024-04-30T06:25:00.000Z', 'actual_start_time': '2024-04-30T05:55:00.000Z', 'promo_codes': {'A2V15ACRFYYP1N', 'A38APJBKCZ6TM4'}} : [50.0] : https://www.amazon.it/dp/B0CB5J5GCV
B0CFFH38N5 : {'start_time': '2024-04-30T06:40:00

In [27]:
for el in db_hash.values():
  promo_details_list = []
  for promo_id in el['promo_codes']:
    promo_details = get_promo_details(promo_id)
    promo_details_list.append(promo_details)
    #el['promo_details'] = promo_details
    print(promo_details)
  el['promo_details'] = promo_details_list

{'percent': '8', 'amount': ''}
{'percent': '10', 'amount': ''}
{'percent': '0', 'amount': '30.00'}
{'percent': '10', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '15', 'amount': ''}
{'percent': '12', 'amount': ''}
{'percent': '20', 'amount': ''}
{'percent': '0', 'amount': '3.00'}
{'percent': '5', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '50', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '20', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '15', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '10', 'amount': ''}
{'percent': '15', 'amount': ''}
{'percent': '5', 'amount': ''}
{'percent': '10', 'amount': ''}
{'percent': '15', 'amount': ''}


KeyboardInterrupt: 

In [ ]:
#questo metodo funzionva, ma a caso da selenium non si trova più l'url "/promo/", dal sorgente chrome c'è ancora
test_product_url = "https://www.amazon.it/dp/B0C2V1113J"
#driver.get(test_product_url)
#html = driver.page_source
#to future me, i'm not stupid, i cannot use response.content.text because for some reason it remove some parts of the hmtl response, exactly the part that we need here
while True
res = requests.get(test_product_url)
html = res.content.decode(res.encoding)
pattern = r'/promo/[^"]*'
promos = re.findall(pattern, html)
[promo.split("/")[-1] for promo in promos]

[]